# Агрегатные функции

## 1. Знакомимся с данными

В этом модуле мы познакомимся с агрегатными функциями.

И помогут нам в этом... покемоны! Как? Да очень просто!

На протяжении всего модуля мы будем работать с таблицей sql.pokemon, содержащей данные о покемонах и их характеристиках из классических видеоигр.

Давайте познакомимся с нашими покемонами и данными о них!

id 	уникальный идентификатор
name 	имя
type1 	основной тип
type2 	дополнительный тип
hp 	количество очков здоровья
attack 	показатели атаки
defense 	показатели защиты
speed 	показатели скорости

## 2. Убираем повторяющиеся значения

Для начала получим все основные типы покемонов.

SELECT
    type1
FROM sql.pokemon

Видно, что типы повторяются, потому что в результате запроса вы получаете все строки, которые подходят под заданные условия.

Чтобы получить уникальные значения из столбца, воспользуемся ключевым словом DISTINCT.

SELECT DISTINCT
    type1
FROM sql.pokemon

SELECT DISTINCT
    type1,
    type2
FROM sql.pokemon

## 3. Агрегатные функции

Кроме простых математических операций, которые мы использовали в предыдущем модуле, СУБД позволяет проводить статистические вычисления для нескольких строк.

Давайте посчитаем количество строк в таблице. Для этого применим агрегатную функцию COUNT.

SELECT
    COUNT(*)
FROM sql.pokemon

COUNT считает строки, а звёздочка (*) в аргументе функции означает, что считаются все строки, которые возвращает запрос.

Внутри функции COUNT мы можем также применять DISTINCT, чтобы вычислить количество уникальных значений.

SELECT /*выбор*/
    COUNT(DISTINCT type1) /*функция подсчёта строк; уникальные значения столбца type1*/
FROM sql.pokemon /*из таблицы sql.pokemon*/

### Основные агрегатные функции

Назовём основные агрегатные функции, с которыми нам предстоит работать:

        COUNT — вычисляет число непустых строк;
        SUM — вычисляет сумму;
        AVG — вычисляет среднее;
        MAX — вычисляет максимум;
        MIN — вычисляет минимум.

Разумеется, вы можете использовать в запросе фильтрацию строк с помощью WHERE, чтобы получить агрегированное значение только для отдельных строк.

SELECT
avg(hp)
FROM sql.pokemon
where type1 = 'Dragon' 

Кроме того, мы можем применять несколько агрегатных функций в одном запросе.

Напишите запрос, который выведет:

    количество покемонов (столбец pokemon_count);
    среднюю скорость (столбец avg_speed);
    максимальное и минимальное число очков здоровья (столбцы max_hp и min_hp)

    для электрических (Electric) покемонов, имеющих дополнительный тип и показатели атаки или защиты больше 50.

SELECT
    COUNT(*) pokemon_count,
    AVG(speed) avg_speed,
    MAX(hp) max_hp,
    MIN(hp) min_hp
FROM sql.pokemon
WHERE type1 = 'Electric'
    AND type2 IS NOT NULL
    AND (attack > 50 OR defense > 50)

## 4. Группировка

GROUP BY используется для определения групп выходных строк, к которым могут применяться агрегатные функции.

SELECT /*выбор*/
    type1 AS pokemon_type, /*столбец type1; присвоить алиас pokemon_type*/
    COUNT(*) AS pokemon_count /*подсчёт всех строк; присвоить алиас pokemon_count*/
FROM sql.pokemon /*из таблицы sql.pokemon*/
GROUP BY type1 /*группировка по столбцу type1*/
ORDER BY type1 /*сортировка по столбцу type1*/

Вывод, конечно же, можно сортировать по столбцу с агрегированием.

SELECT /*выбор*/
    type1 AS pokemon_type, /*столбец type1; присвоить алиас pokemon_type*/
    COUNT(*) AS pokemon_count /*подсчёт всех строк; присвоить алиас pokemon_count*/
FROM sql.pokemon /*из таблицы sql.pokemon*/
GROUP BY pokemon_type /*группировка по столбцу pokemon_type*/
ORDER BY COUNT(*) DESC /*сортировка в порядке убывания*/

__
Напишите запрос, который выведет:

    число различных дополнительных типов (столбец additional_types_count);
    среднее число очков здоровья (столбец avg_hp);
    сумму показателей атаки (столбец attack_sum) в разбивке по основным типам (столбец primary_type).

Отсортируйте результат по числу дополнительных типов в порядке убывания, при равенстве — по основному типу в алфавитном порядке.
Столбцы к выводу (обратите внимание на порядок!): primary_type, additional_types_count, avg_hp, attack_sum.

SELECT
   type1 primary_type,
   count(DISTINCT type2) additional_types_count,
   AVG(hp) avg_hp,
   SUM(attack) attack_sum
FROM sql.pokemon
GROUP BY primary_type
ORDER BY 2 DESC, 1

Мы можем осуществлять группировку по нескольким столбцам.

SELECT /*выбор*/
    type1 AS primary_type, /*столбец type1; присвоить алиас primary_type*/
    type2 AS additional_type, /*столбец type2; присвоить алиас additional_type*/
    COUNT(*) AS pokemon_count /*подсчёт всех строк присвоить алиас pokemon_count*/
FROM sql.pokemon /*из таблицы sql.pokemon*/
GROUP BY 1, 2 /*группировка по столбцам 1 и 2*/
ORDER BY 1, 2 NULLS FIRST /*сортировка по столбцам 1 и 2; сначала нули*/

Обратите внимание! В группировке можно указывать порядковый номер столбца так же, как мы делали это в прошлом модуле для сортировки.

## 5. Фильтрация агрегированных строк

Если ключевое слово WHERE определяет фильтрацию строк до агрегирования, то для фильтрации уже агрегированных данных применяется ключевое слово HAVING.

Важно! HAVING обязательно пишется после GROUP BY.

Выведем типы покемонов и их средний показатель атаки, при этом оставим только тех, у кого средняя атака больше 90:

SELECT /*выбор*/
    type1 AS primary_type, /*таблица type1; присвоить алиас primary_type*/
    AVG(attack) AS avg_attack /*расчёт среднего по столбцу attack; присвоить алиас avg_attack*/
FROM sql.pokemon /*из таблицы sql.pokemon*/
GROUP BY primary_type /*группировать по столбцу primary_type*/
HAVING AVG(attack) > 90 /*фильтровать по среднему значению attack, превышающему 90*/

Вместо резюме

В общем виде синтаксис оператора SELECT, с учётом имеющихся на данный момент знаний, представляем следующим образом:

SELECT [ALL | DISTINCT] список_столбцов|*
FROM список_имён_таблиц
[WHERE условие_поиска]
[GROUP BY список_имён_столбцов]
[HAVING условие_поиска]
[ORDER BY имя_столбца [ASC | DESC],…]

Обратите внимание! В квадратных скобках указаны необязательные предложения: они могут отсутствовать в операторе SELECT.

Задания

1. Напишите запрос, который выведет основной и дополнительный типы покемонов (столбцы primary_type и additional_type) для тех типов, у которых средний показатель атаки больше 100 и максимальный показатель очков здоровья меньше 80.

SELECT
    type1 primary_type,
    type2 additional_type
FROM sql.pokemon
GROUP BY primary_type, additional_type
HAVING avg(attack) > 100
    AND MAX(hp) < 80


2. Напишите запрос, который выводит столбцы с основным типом (primary_type) и числом покемонов этого типа (pokemon_count) для тех покемонов, чьё имя (name) начинается с S.

Оставьте только типы, у которых средний показатель защиты больше 80.

Выведите только ТОП-3 типов по числу покемонов в них.

SELECT
    type1 primary_type,
    count(*) pokemon_count
FROM sql.pokemon
where name like 'S%'
GROUP BY primary_type
HAVING avg(defense) > 80
ORDER BY pokemon_count desc
LIMIT 3

И в довершение итогов напомним структуру запроса, который мы можем составить с учётом новых знаний:

SELECT
    столбец1 AS новое_название,
    столбец2,
    АГРЕГАТ(столбец3)
FROM таблица
WHERE (условие1 OR условие2)
    AND условие3
GROUP BY столбец1, столбец2
HAVING АГРЕГАТ(столбец3) > 5
ORDER BY сортировка1, сортировка2
OFFSET 1 LIMIT 2

Напишите запрос, который выведет основной и дополнительный типы покемонов и средние значения по каждому показателю (столбцы avg_hp, avg_attack, avg_defense, avg_speed).
Оставьте только те пары типов, у которых сумма этих четырёх показателей более 400.

select 
    type1 AS primary_type,
    type2 AS additional_type,
    avg(hp) AS avg_hp,
    avg(attack) AS avg_attack,
    avg(defense) AS avg_defense,
    avg(speed) AS avg_speed    
from sql.pokemon
group by primary_type, additional_type
having (avg(hp) + avg(attack) + avg(defense) + avg(speed)) > 400


Напишите запрос, который выведет столбцы с основным типом покемона и общим количеством покемонов этого типа.

Учитывайте только тех покемонов, у которых или показатель атаки, или показатель защиты принимает значение между 50 и 100 включительно.

Оставьте только те типы покемонов, у которых максимальный показатель здоровья не больше 125.

Выведите только тот тип, который находится на пятом месте по количеству покемонов.

SELECT
    type1 AS primary_type,
    count(*) AS pokemon_count
FROM sql.pokemon
where (attack between 50 and 100)
    or (defense between 50 and 100)
GROUP BY primary_type
HAVING max(hp) <= 125
ORDER BY COUNT(*) DESC
OFFSET 4 LIMIT 1
